In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import r2_score

vars_to_plot = [
    "ros_p_lg1", "ros_p_lg2", "ros_p_lg3",
    'BLH_m_av', 'BLH_m_mn', 'BLH_m_mx',
    'dfmc_av', 'dfmc_mn', 'dfmc_mx',
    'wv100_k_av', 'wv100_k_mn', 'wv100_k_mx',
    "wS_v_av_1", "wS_v_mn_1", "wS_v_mx_1",
    'Recirc', 'CircVar', 'CircStd_dg',
    'd_2m_C_av', 'd_2m_C_mn', 'd_2m_C_mx',
    "HDW_av", 'HDW_mn', 'HDW_mx',
    'LCL_hPa_av', 'LCL_hPa_mn', 'LCL_hPa_mx',
    "wv_850_av", 'wv_850_mn', 'wv_850_mx',
    'wv10_kh_av', 'wv10_kh_mn', 'wv10_kh_mx',
]

# Store results
results = []

print("="*80)
print("BEST TRANSFORMATION SELECTION (by R²)")
print("="*80)

# Loop through all variables
for var in vars_to_plot:
    if var not in df.columns or var not in df_transformed.columns:
        print(f"\n{var}: NOT FOUND in both dataframes, skipping.")
        continue
    
    if 'ros_p' not in df.columns or 'ros_p' not in df_transformed.columns:
        print(f"\nros_p column not found in both dataframes, stopping.")
        break
    
    # Get linear data
    x_linear = pd.to_numeric(df[var], errors='coerce')
    y_linear = pd.to_numeric(df['ros_p'], errors='coerce')
    
    # Get log-transformed data
    x_log = pd.to_numeric(df_transformed[var], errors='coerce')
    y_log = pd.to_numeric(df_transformed['ros_p'], errors='coerce')
    
    # Define the four combinations
    combinations = [
        ('Linear-Linear', x_linear, y_linear),
        ('Linear-LOG', x_linear, y_log),
        ('LOG-Linear', x_log, y_linear),
        ('LOG-LOG', x_log, y_log)
    ]
    
    # Calculate R² for each combination
    r2_values = {}
    r_values = {}
    p_values = {}
    n_samples = {}
    
    for name, x, y in combinations:
        # Remove NaN values
        mask = x.notna() & y.notna()
        x_clean = x[mask]
        y_clean = y[mask]
        
        if len(x_clean) < 5:
            r2_values[name] = -999  # Sentinel value for insufficient data
            r_values[name] = np.nan
            p_values[name] = np.nan
            n_samples[name] = len(x_clean)
            continue
        
        # Linear fit
        coeffs = np.polyfit(x_clean, y_clean, 1)
        y_pred = np.polyval(coeffs, x_clean)
        r2 = r2_score(y_clean, y_pred)
        
        # Pearson correlation
        r, p_value = stats.pearsonr(x_clean, y_clean)
        
        r2_values[name] = r2
        r_values[name] = r
        p_values[name] = p_value
        n_samples[name] = len(x_clean)
    
    # Find the best transformation
    valid_r2 = {k: v for k, v in r2_values.items() if v > -999}
    
    if not valid_r2:
        print(f"\n{var}: INSUFFICIENT DATA for all transformations")
        continue
    
    best_transform = max(valid_r2, key=valid_r2.get)
    best_r2 = r2_values[best_transform]
    best_r = r_values[best_transform]
    best_p = p_values[best_transform]
    best_n = n_samples[best_transform]
    
    # Print results
    print(f"\n{var} vs ros_p:")
    print(f"  Best transformation: {best_transform}")
    print(f"  R² = {best_r2:.4f} | r = {best_r:.4f} | p = {best_p:.2e} | n = {best_n}")
    print(f"  All R² values:")
    for trans_name in ['Linear-Linear', 'Linear-LOG', 'LOG-Linear', 'LOG-LOG']:
        r2_val = r2_values[trans_name]
        if r2_val > -999:
            marker = " ← BEST" if trans_name == best_transform else ""
            print(f"    {trans_name}: R² = {r2_val:.4f}{marker}")
        else:
            print(f"    {trans_name}: INSUFFICIENT DATA")
    
    # Store for summary table
    results.append({
        'Variable': var,
        'Best_Transformation': best_transform,
        'R²': best_r2,
        'r': best_r,
        'p-value': best_p,
        'n': best_n
    })

# Create summary DataFrame
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('R²', ascending=False)

print(results_df.to_string(index=False))

# Count transformations
print("\n" + "="*80)
print("TRANSFORMATION FREQUENCY")
print("="*80)
transform_counts = results_df['Best_Transformation'].value_counts()
for trans, count in transform_counts.items():
    print(f"{trans}: {count} variables ({count/len(results_df)*100:.1f}%)")

print("\n" + "="*80)

In [ ]:
### Normality tests - Q-Q Plots

In [ ]:
def assess_normality_visual(df, columns, n_cols=3):
    n_features = len(columns)
    if n_features == 0:
        print("No columns to assess.")
        return
    
    n_rows = (n_features + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    
    # Flatten axes for easy iteration
    axes = axes.flatten() if n_features > 1 else [axes]
    
    for idx, col in enumerate(columns):
        data = df[col].dropna()
        if len(data) < 5:
            axes[idx].axis('off')
            continue

        # Q-Q plot
        stats.probplot(data, dist="norm", plot=axes[idx])
        axes[idx].set_title(f'{col}\n(skew: {data.skew():.2f}, kurt: {data.kurtosis():.2f})')
        axes[idx].grid(True, alpha=0.3)
    
    # Turn off unused subplots
    for idx in range(len(columns), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()


# --- Apply to all transformed DataFrames ---
for name, df_transformed in transformed_dataframes.items():
    print(f"\nAssessing normality for transformed columns in {name}...")

    if name not in transformation_logs:
        print(f"No transformation log for {name}, skipping.")
        continue

    # Select columns that were transformed using Yeo-Johnson
    transformed_cols = [col for col, status in transformation_logs[name].items()
                        if 'Yeo-Johnson' in status]

    if not transformed_cols:
        print(f"No Yeo-Johnson transformed columns found in {name}.")
        continue

    assess_normality_visual(df_transformed, transformed_cols)

In [ ]:
### Normality - hypothesis testing

In [ ]:
def run_normality_pipeline(df, df_transformed, dataset_name="Dataset", alpha=0.05):
    print("="*80)
    print(f"NORMALITY TESTING FOR {dataset_name.upper()}")
    print(f"Original shape: {df.shape}, Transformed shape: {df_transformed.shape}")
    print("="*80)

    # --- 1. Run normality tests on transformed data ---
    results = []
    numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns

    for col in tqdm(numeric_cols, desc="Normality testing"):
        data = df_transformed[col].dropna()
        if len(data) < 3:
            continue
        
        shapiro_stat, shapiro_p = stats.shapiro(data)
        jb_stat, jb_p = stats.jarque_bera(data)
        skewness = stats.skew(data)
        kurtosis = stats.kurtosis(data)
        dagostino_stat, dagostino_p = stats.normaltest(data)

        results.append({
            'Column': col,
            'N': len(data),
            'Mean': data.mean(),
            'Std': data.std(),
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'Shapiro_W': shapiro_stat,
            'Shapiro_p': shapiro_p,
            'JB_p': jb_p,
            'DAgostino_p': dagostino_p,
            'Normal_Shapiro': 'Yes' if shapiro_p > alpha else 'No',
            'Normal_JB': 'Yes' if jb_p > alpha else 'No',
            'Normal_DAgostino': 'Yes' if dagostino_p > alpha else 'No'
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'normality_results_{dataset_name}.csv', index=False)
    print(f"\n✓ Detailed normality results saved: normality_results_{dataset_name}.csv")

    # --- 2. Summary ---
    total_cols = len(results_df)
    shapiro_normal = (results_df['Shapiro_p'] > alpha).sum()
    jb_normal = (results_df['JB_p'] > alpha).sum()
    dagostino_normal = (results_df['DAgostino_p'] > alpha).sum()
    all_normal = ((results_df['Shapiro_p'] > alpha) & 
                  (results_df['JB_p'] > alpha) & 
                  (results_df['DAgostino_p'] > alpha)).sum()
    all_non_normal = ((results_df['Shapiro_p'] <= alpha) & 
                      (results_df['JB_p'] <= alpha) & 
                      (results_df['DAgostino_p'] <= alpha)).sum()

    print(f"\nTotal numeric features tested: {total_cols}")
    print(f"Shapiro-Wilk normal: {shapiro_normal} ({shapiro_normal/total_cols*100:.1f}%)")
    print(f"Jarque-Bera normal: {jb_normal} ({jb_normal/total_cols*100:.1f}%)")
    print(f"D'Agostino-Pearson normal: {dagostino_normal} ({dagostino_normal/total_cols*100:.1f}%)")
    print(f"All tests agree NORMAL: {all_normal} ({all_normal/total_cols*100:.1f}%)")
    print(f"All tests agree NON-NORMAL: {all_non_normal} ({all_non_normal/total_cols*100:.1f}%)")
    print(f"Tests disagree: {total_cols - all_normal - all_non_normal}")

    # --- 3. Overview plots ---
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    axes[0].hist(results_df['Shapiro_p'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].axvline(alpha, color='red', linestyle='--', label=f'α={alpha}')
    axes[0].set_title('Shapiro-Wilk P-value Distribution'); axes[0].legend()

    axes[1].hist(results_df['JB_p'], bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(alpha, color='red', linestyle='--', label=f'α={alpha}')
    axes[1].set_title('Jarque-Bera P-value Distribution'); axes[1].legend()

    axes[2].hist(results_df['DAgostino_p'], bins=50, edgecolor='black', alpha=0.7)
    axes[2].axvline(alpha, color='red', linestyle='--', label=f'α={alpha}')
    axes[2].set_title("D'Agostino-Pearson P-value Distribution"); axes[2].legend()

    axes[3].hist(results_df['Skewness'], bins=50, edgecolor='black', alpha=0.7)
    axes[3].axvline(0, color='red', linestyle='--'); axes[3].set_title('Skewness Distribution')

    axes[4].hist(results_df['Kurtosis'], bins=50, edgecolor='black', alpha=0.7)
    axes[4].axvline(0, color='red', linestyle='--'); axes[4].set_title('Kurtosis Distribution')

    colors = ['green' if p > alpha else 'red' for p in results_df['Shapiro_p']]
    axes[5].scatter(results_df['Skewness'], results_df['Kurtosis'], c=colors, alpha=0.5, s=20)
    axes[5].axhline(0, color='gray', linestyle='--'); axes[5].axvline(0, color='gray', linestyle='--')
    axes[5].set_title('Skewness vs Kurtosis (Green=Normal, Red=Non-normal)')

    plt.suptitle(f'Normality Overview - {dataset_name}', fontsize=16)
    plt.tight_layout(rect=[0,0,1,0.95])
    plt.show()

    return results_df

# Loop through all transformed datasets
for name, df_transformed in transformed_dataframes.items():
    run_normality_pipeline(df, df_transformed, dataset_name=name)

In [ ]:
# Loop through all transformed DataFrames
for name, df_trans in transformed_dataframes.items():
    if 'ros_p' not in df_trans.columns:
        print(f"Skipping {name} — 'ros_p' not found.")
        continue

    ros_data = pd.to_numeric(df_trans['ros_p'], errors='coerce').dropna()

    if len(ros_data) < 3:
        print(f"Skipping {name} — not enough valid ros_p data.")
        continue

    # Compute mean and std
    mu, sigma = ros_data.mean(), ros_data.std()

    # Plot histogram with normal overlay
    plt.figure(figsize=(10, 6))
    plt.hist(ros_data, bins=100, density=True, alpha=0.6, color='steelblue', edgecolor='black')

    # Normal PDF
    x = np.linspace(ros_data.min(), ros_data.max(), 1000)
    pdf = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, pdf, 'r', linewidth=2, label=f'N({mu:.2f}, {sigma:.2f}²)')

    # Labels and title
    plt.title(f"Distribution of ROS (ros_p) — {name}")
    plt.xlabel("ROS (ros_p)")
    plt.ylabel("Density")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    # D’Agostino–Pearson test
    stat, p_value = stats.normaltest(ros_data)
    print(f"{'-'*60}\nDataset: {name}")
    print("D’Agostino–Pearson Normality Test for ros_p")
    print(f"Statistic: {stat:.4f}")
    print(f"p-value: {p_value:.4e}")
    if p_value > 0.05:
        print("Fail to reject H₀ — data appears approximately normal.")
    else:
        print("Reject H₀ — data likely not normal.")

In [ ]:
# SCATTER PLOT das correlações com ros_p (top 10 legendadas, invertido)

# Columns to exclude from correlation
exclude_cols = [
    'fid', 'fname', 'year', 'id', 'type', 'sdate', 'edate',
    'inidoy', 'enddoy', 'source', 'zp_link', 'burn_perio', 'area',
    'growth_rat', 'ros_i', 'ros_p', 'spdir_i', 'spdir_p',
    'int_i', 'int_p', 'duration_i', 'duration_p', 'qc',
    '1_3y_fir_p', '3_8y_fir_p', '8_ny_fir_p', "ros_p_lag1", "ros_p_lag2"
]

# Select numeric columns except the excluded ones
numeric_subset = df_transformed.select_dtypes(include=[np.number])
numeric_subset = numeric_subset.drop(columns=[c for c in exclude_cols if c in numeric_subset.columns], errors='ignore')

# Make sure ros_p is included for correlation
numeric_subset = pd.concat([numeric_subset, df_transformed[['ros_p']]], axis=1)

# 1️Compute correlation of all allowed variables with 'ros_p'
corr_with_ros = numeric_subset.corr()['ros_p'].drop('ros_p')

# 2️Sort by absolute value of correlation
corr_sorted = corr_with_ros.reindex(corr_with_ros.abs().sort_values(ascending=False).index)

# 3️Take top 20 for labeling
top20_vars = corr_sorted.head(10).index

# 4️Scatter plot
plt.figure(figsize=(12, 6))
for i, var in enumerate(corr_sorted.index):
    alpha = 1.0 if var in top20_vars else 0.1
    color = 'royalblue' if corr_sorted[var] >= 0 else 'tomato'
    plt.scatter(i, corr_sorted[var], color=color, alpha=alpha, s=60)

# 5️Label only top 20
for i, var in enumerate(corr_sorted.index):
    if var in top20_vars:
        plt.text(i, corr_sorted[var], var, fontsize=9, ha='right', va='bottom', rotation=45)

plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.gca().invert_xaxis()
plt.title("Correlation with ros_p", fontsize=14)
plt.ylabel("Pearson Correlation Coefficient")
plt.xticks([])
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

# Mostrar top 20 correlações com ros_p em DataFrame

top_corr_df = corr_sorted.head(10).reset_index()
top_corr_df.columns = ['Variable', 'correlation_with_ros_p']

# Ordenar: positivos (desc) primeiro, depois negativos (asc)
top_corr_df = pd.concat([
    top_corr_df[top_corr_df['correlation_with_ros_p'] > 0].sort_values(by='correlation_with_ros_p', ascending=False),
    top_corr_df[top_corr_df['correlation_with_ros_p'] < 0].sort_values(by='correlation_with_ros_p', ascending=True)
]).reset_index(drop=True)

display(top_corr_df)

In [ ]:
### Test correlation with ros_p before and after transformation and lists variables that have less cor after

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import normaltest # New import for D'Agostino-Pearson
from sklearn.preprocessing import PowerTransformer
import warnings
from sklearn.exceptions import DataConversionWarning

# --- Assumes your DataFrame 'df' is loaded here ---

## Data Preparation and Column Identification
# Identify columns that are purely numeric for testing and transformation
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
cols_to_test_and_transform = [col for col in numeric_cols if col != 'ros_p']
cols_to_exclude = [col for col in df.columns if col not in numeric_cols] + ['ros_p']

print(f"Total columns found: {len(df.columns)}")
print(f"Numeric columns to process: {cols_to_test_and_transform}")
print(f"Columns excluded: {cols_to_exclude}")
print("-" * 50)


## 1. Initial Normality Tests (D’Agostino–Pearson)

print("--- 1. Normality Tests Before Transformation (D'Agostino-Pearson) ---")
results_before = []
for col in cols_to_test_and_transform:
    data = df[col].dropna()
    # D'Agostino-Pearson test requires at least 8 non-null samples (N>=20 is recommended)
    if len(data) >= 8:
        stat, p = normaltest(data)
        results_before.append({'Variable': col, 'DAP_Stat_Before': stat, 'P_Value_Before': p})
    else:
        results_before.append({'Variable': col, 'DAP_Stat_Before': np.nan, 'P_Value_Before': np.nan})

results_before_df = pd.DataFrame(results_before)
print(results_before_df)
print("-" * 50)


## 2. Yeo-Johnson Transformation

df_transformed = df.copy()

if cols_to_test_and_transform:
    print(f"Transforming variables: {', '.join(cols_to_test_and_transform)}")

    warnings.filterwarnings(action='ignore', category=DataConversionWarning)
    # Initialize PowerTransformer for Yeo-Johnson
    pt = PowerTransformer(method='yeo-johnson', standardize=False)

    try:
        # Fit and transform the specified columns
        df_transformed[cols_to_test_and_transform] = pt.fit_transform(df[cols_to_test_and_transform])

        print("--- Transformed DataFrame Head (subset) ---")
        print(df_transformed[cols_to_test_and_transform[:3] + ['ros_p']].head()) # Show a subset + ros_p
        print("-" * 50)

    except ValueError as e:
        print(f"Error during transformation: {e}")
        # Continue with post-test on original data if transform fails
        df_transformed = df.copy()


## 3. Post-Transformation Normality Tests (D’Agostino–Pearson)

print("--- 3. Normality Tests After Transformation (D'Agostino-Pearson) ---")
results_after = []
for col in cols_to_test_and_transform:
    data = df_transformed[col].dropna()
    if len(data) >= 8:
        stat, p = normaltest(data)
        results_after.append({'Variable': col, 'DAP_Stat_After': stat, 'P_Value_After': p})
    else:
        results_after.append({'Variable': col, 'DAP_Stat_After': np.nan, 'P_Value_After': np.nan})

results_after_df = pd.DataFrame(results_after)
print(results_after_df)
print("-" * 50)


## 4. Compare Results and List Variables

print("--- 4. Comparison of Normality (Before vs. After) ---")

# Merge the results
all_results = pd.merge(results_before_df, results_after_df, on='Variable')

# Calculate the change in p-value
all_results['P_Value_Change'] = all_results['P_Value_After'] - all_results['P_Value_Before']

print("Full comparison of transformed variables:")
print(all_results[['Variable', 'P_Value_Before', 'P_Value_After', 'P_Value_Change']].to_string())
print("-" * 50)

# Find variables where normality DECREASED
# This means the p-value became smaller (moving *away* from normality)
decreased_normality_vars = all_results[
    all_results['P_Value_After'] < all_results['P_Value_Before']
]

print("================================================================")
print("🔴 VARIABLES THAT DECREASED IN NORMALITY (P-Value went DOWN)")
print("================================================================")
if decreased_normality_vars.empty:
    print("None of the transformed variables showed a decrease in normality.")
else:
    print(decreased_normality_vars[['Variable', 'P_Value_Before', 'P_Value_After']])

print("================================================================")